# DAT-DEN-03 | Demo 14 | Latent Variable Modeling with `gensim`

`gensim` (http://radimrehurek.com/gensim) is a library of language processing tools focused on latent variable models of text.

In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from gensim import matutils, models

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

%matplotlib inline
plt.style.use('ggplot')

The data is about sentiments on Amazon reviews.

In [2]:
reviews = []
sentiments = []

with open(os.path.join('..', 'datasets', 'amazon-reviews.txt')) as f:
    for line in f.readlines():
        line = line.strip('\n')
        review, sentiment = line.split('\t')
        sentiment = np.nan if sentiment == '' else int(sentiment)

        reviews.append(review.lower())
        sentiments.append(sentiment)

df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [3]:
df.head()

,review,sentiment
0,i try not to adjust the volume setting to avoi...,NaN
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
3,i thought motorola made reliable products!.,NaN
4,battery for motorola razr.,NaN


In [4]:
df.dropna(inplace = True) # Let's drop the NaN

In [5]:
df.head()

,review,sentiment
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
5,great for the jawbone.,1.0
10,tied to charger for conversations lasting more...,0.0
11,the mic is great.,1.0


## LDA with `gensim`

### Let's first translate a set of documents (articles) into a matrix representation with a row per document and a column per feature (word or n-gram)

In [6]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words = 'english')

In [7]:
documents = vectorizer.fit_transform(df.review)

In [8]:
# Let's now build a mapping of numerical ID to word

id2word = dict(enumerate(vectorizer.get_feature_names()))

In [9]:
id2word

{0: u'10',
 1: u'100',
 2: u'11',
 3: u'12',
 4: u'13',
 5: u'15',
 6: u'15g',
 7: u'18',
 8: u'20',
 9: u'2000',
 10: u'2005',
 11: u'2160',
 12: u'24',
 13: u'2mp',
 14: u'325',
 15: u'350',
 16: u'375',
 17: u'3o',
 18: u'42',
 19: u'44',
 20: u'45',
 21: u'4s',
 22: u'50',
 23: u'5020',
 24: u'510',
 25: u'5320',
 26: u'680',
 27: u'700w',
 28: u'8125',
 29: u'8525',
 30: u'8530',
 31: u'abhor',
 32: u'ability',
 33: u'able',
 34: u'abound',
 35: u'absolutel',
 36: u'absolutely',
 37: u'ac',
 38: u'accept',
 39: u'acceptable',
 40: u'access',
 41: u'accessable',
 42: u'accessing',
 43: u'accessory',
 44: u'accessoryone',
 45: u'accidentally',
 46: u'accompanied',
 47: u'according',
 48: u'activate',
 49: u'activated',
 50: u'activesync',
 51: u'actually',
 52: u'ad',
 53: u'adapter',
 54: u'adapters',
 55: u'add',
 56: u'addition',
 57: u'additional',
 58: u'address',
 59: u'adhesive',
 60: u'adorable',
 61: u'advertised',
 62: u'advise',
 63: u'aggravating',
 64: u'ago',
 65: u'al

### We want to learn which columns are correlated (i.e., likely to come from the same topic).  This is the word distribution.  We can also determine what topics are in each document, the topic distribution.

In [10]:
# First we convert our word-matrix into gensim's format

corpus = matutils.Sparse2Corpus(documents, documents_columns = False)

(Check https://radimrehurek.com/gensim/matutils as needed)

In [11]:
corpus

(Check https://radimrehurek.com/gensim/models/ldamodel as needed)

In [12]:
# Then we fit an LDA model

model = models.ldamodel.LdaModel(corpus = corpus, num_topics = 25, id2word = id2word, passes = 10)

In this model, we need to explicitly specify the number of topic we want the model to uncover.  This is a critical parameter, but there isn't much guidance on how to choose it.  Try to use domain expertise where possible.

In [13]:
model

### Goodness of fit

Now we need to assess the goodness of fit for our model.  Like other unsupervised learning techniques, our validation techniques are mostly about interpretation.

Use the following questions to guide you:
- Did we learn reasonable topics?
- Do the words that make up a topic make sense?
- Is this topic helpful towards our goal?

In [14]:
model.print_topics()

[(11,
  u'0.099*"phone" + 0.089*"great" + 0.023*"good" + 0.021*"worst" + 0.021*"device" + 0.019*"ve" + 0.013*"nice" + 0.013*"samsung" + 0.013*"new" + 0.010*"camera"'),
 (22,
  u'0.033*"use" + 0.033*"easy" + 0.021*"sound" + 0.017*"phone" + 0.013*"reception" + 0.013*"battery" + 0.013*"difficult" + 0.013*"sucks" + 0.012*"cell" + 0.009*"great"'),
 (8,
  u'0.023*"ve" + 0.023*"long" + 0.018*"couldn" + 0.016*"good" + 0.015*"bargain" + 0.014*"quality" + 0.014*"bought" + 0.014*"sending" + 0.013*"fast" + 0.011*"phone"'),
 (18,
  u'0.066*"recommend" + 0.044*"headset" + 0.035*"phone" + 0.031*"best" + 0.030*"highly" + 0.027*"bad" + 0.024*"ve" + 0.017*"love" + 0.017*"months" + 0.014*"think"'),
 (3,
  u'0.048*"phone" + 0.025*"great" + 0.024*"good" + 0.014*"sturdy" + 0.014*"disappointment" + 0.014*"audio" + 0.011*"don" + 0.011*"problems" + 0.011*"gets" + 0.011*"big"'),
 (17,
  u'0.060*"phone" + 0.030*"horrible" + 0.030*"love" + 0.020*"problem" + 0.019*"use" + 0.015*"like" + 0.014*"purchase" + 0.012*"b

Some topics will be clearer than others.  The following topics represent clear concepts:
- Cooking and Recipes: 0.009 \* cup + 0.009 \* recipe + 0.007 \* make + 0.007 \* food + 0.006 \* sugar
- Cooking and recipes: 0.013 \* butter + 0.010 \* baking + 0.010 \* dough + 0.009 \* cup + 0.009 \* sugar
- Fashion and Style: 0.013 \* fashion + 0.006 \* like + 0.006 \* dress + 0.005 \* style

## Word2Vec with `gensim`

In [15]:
# Setup the body text
sentences = df.review.map(lambda review: review.split())

In [16]:
sentences

1       [so, there, is, no, way, for, me, to, plug, it...
2                        [good, case,, excellent, value.]
5                             [great, for, the, jawbone.]
10      [tied, to, charger, for, conversations, lastin...
11                                 [the, mic, is, great.]
                              ...                        
2925    [the, screen, does, get, smudged, easily, beca...
2930    [what, a, piece, of, junk.., i, lose, more, ca...
2934                   [item, does, not, match, picture.]
2935    [the, only, thing, that, disappoint, me, is, t...
2937    [you, can, not, answer, calls, with, the, unit...
Name: review, Length: 1000, dtype: object

In [17]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

`Word2Vec` has many arguments:
- `size` represents how many concepts or topics we should use
- `window` represents how many words surrounding a sentence we should use as our original feature
- `min_count` is the number of times that context or word must appear
- `workers` is the number of CPU cores to use to speed up model training

(Check http://radimrehurek.com/gensim/models/word2vec as needed)

In [18]:
model

### Most similar words

The model has a `most_similar` function that helps find the words most similar to the one you queried.  This will return words that are most often used in the same context.

In [19]:
model.most_similar(positive = ['great'])

[('to', 0.9989875555038452),
 ('out', 0.9989862442016602),
 ('my', 0.9989322423934937),
 ('the', 0.9988909959793091),
 ('it', 0.9988875389099121),
 ('this', 0.9988873600959778),
 ('i', 0.9988756775856018),
 ('for', 0.9988580346107483),
 ('a', 0.998855710029602),
 ('and', 0.998836874961853)]

In [20]:
vectorizer.get_feature_names()

[u'10',
 u'100',
 u'11',
 u'12',
 u'13',
 u'15',
 u'15g',
 u'18',
 u'20',
 u'2000',
 u'2005',
 u'2160',
 u'24',
 u'2mp',
 u'325',
 u'350',
 u'375',
 u'3o',
 u'42',
 u'44',
 u'45',
 u'4s',
 u'50',
 u'5020',
 u'510',
 u'5320',
 u'680',
 u'700w',
 u'8125',
 u'8525',
 u'8530',
 u'abhor',
 u'ability',
 u'able',
 u'abound',
 u'absolutel',
 u'absolutely',
 u'ac',
 u'accept',
 u'acceptable',
 u'access',
 u'accessable',
 u'accessing',
 u'accessory',
 u'accessoryone',
 u'accidentally',
 u'accompanied',
 u'according',
 u'activate',
 u'activated',
 u'activesync',
 u'actually',
 u'ad',
 u'adapter',
 u'adapters',
 u'add',
 u'addition',
 u'additional',
 u'address',
 u'adhesive',
 u'adorable',
 u'advertised',
 u'advise',
 u'aggravating',
 u'ago',
 u'alarm',
 u'allot',
 u'allow',
 u'allowing',
 u'allows',
 u'alot',
 u'aluminum',
 u'amazed',
 u'amazing',
 u'amazon',
 u'amp',
 u'ample',
 u'angeles',
 u'angle',
 u'answer',
 u'ant',
 u'antena',
 u'anti',
 u'apart',
 u'apartment',
 u'apparently',
 u'appeali

In [21]:
sentences

1       [so, there, is, no, way, for, me, to, plug, it...
2                        [good, case,, excellent, value.]
5                             [great, for, the, jawbone.]
10      [tied, to, charger, for, conversations, lastin...
11                                 [the, mic, is, great.]
                              ...                        
2925    [the, screen, does, get, smudged, easily, beca...
2930    [what, a, piece, of, junk.., i, lose, more, ca...
2934                   [item, does, not, match, picture.]
2935    [the, only, thing, that, disappoint, me, is, t...
2937    [you, can, not, answer, calls, with, the, unit...
Name: review, Length: 1000, dtype: object

In [22]:
sentences = list(map(lambda sentence: list(filter(lambda word: word in vectorizer.get_feature_names(), sentence)), sentences))

In [23]:
sentences

[['way', 'plug', 'unless'],
 ['good', 'excellent'],
 ['great'],
 ['tied', 'charger', 'conversations', 'lasting', '45'],
 ['mic'],
 ['jiggle', 'plug', 'line', 'right', 'decent'],
 ['dozen', 'imagine', 'fun', 'sending'],
 ['razr'],
 ['needless', 'wasted'],
 ['waste', 'money'],
 ['sound', 'quality'],
 ['impressed', 'going', 'original', 'battery', 'extended'],
 ['seperated',
  'mere',
  'ft',
  'started',
  'notice',
  'excessive',
  'static',
  'garbled',
  'sound'],
 ['good', 'quality'],
 ['design', 'ear', 'comfortable'],
 ['highly', 'recommend', 'blue', 'tooth'],
 ['advise'],
 ['far'],
 ['works'],
 ['clicks', 'place', 'way', 'makes', 'wonder', 'long', 'mechanism'],
 ['went', 'website', 'followed', 'pair'],
 ['bought', 'use', 'kindle', 'absolutely', 'loved'],
 ['commercials'],
 ['run', 'new', 'battery', 'bars', 'days'],
 ['bought', 'mother', 'problem'],
 ['great', 'pocket', 'pc', 'phone'],
 ['owned', 'phone', 'months', 'say', 'best', 'mobile', 'phone'],
 ['think', 'instructions', 'provid

In [24]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

In [25]:
model.most_similar(positive = ['great'])

[('having', 0.1958898901939392),
 ('just', 0.19557274878025055),
 ('recommend', 0.19267727434635162),
 ('screen', 0.1845075488090515),
 ('headsets', 0.18310217559337616),
 ('came', 0.1788807362318039),
 ('piece', 0.15516793727874756),
 ('headphones', 0.15465229749679565),
 ('definitely', 0.15168102085590363),
 ('fine', 0.14670789241790771)]